In [1]:
'''
Date: 2022.03.23
Title: 
By: Kang Jin Seong
'''
''' ## 교차 검증과 그리드 서치'''

'''### 검증세트 '''

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

# 훈련세트 80%에서 다시 훈련세트 60%, 검증세트 20%로 나누어 검증세트를 준비한다.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

#기존 5,197개였던 훈련세트가 4157개로 줄고 검즘 세트는 1040개가 되었다.
print(sub_input.shape, val_input.shape)


(4157, 3) (1040, 3)


In [2]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련세트에 대한 과대적합

0.9971133028626413
0.864423076923077


In [3]:
''' # 교차 검증 '''
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# cross_validate: 기본적으로 5-폴드 교차 검증을 수행한다.


{'fit_time': array([0.01116729, 0.01015306, 0.01070237, 0.01075339, 0.01020288]), 'score_time': array([0.0013485 , 0.00127149, 0.00123501, 0.00124812, 0.00124145]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [4]:
# 교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.
# 이름은 test_score지만 검증 폴드의 점수이다.
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [5]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않습니다.
# 기존 train_test_split 함수를 이용하여 전체 데이터를 섞은 후 훈련세트를 준비 했기 때문에 따로 섞을 필요가 없었다.
# 교차검증을 할 때 훈련세트를 섞으려면 분할기(spliter)를 지정해야한다.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)  # n_splits: 몇(k)폴드 교차 검증을 할지 정합니다.
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [9]:
'''# 히이퍼파라미터 튜닝 '''
'''##그리드 서치 '''

from sklearn.model_selection import GridSearchCV    # GridSearchCV 클래스는 친절하게도 하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다.
params = {'min_impurity_decrease':[0.0001, 0.002, 0.003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_ # 그리스 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련함 -> 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있다.
print(dt.score(train_input, train_target))
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [10]:
# 각 매개변수에서 수행한 고차 검증의 평균점스는 cs_results_ 속성의 mean_test_score 키에 저장되어 있다.
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

[0.86819297 0.85761105 0.85607111 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [13]:
# 더 복잡한 매개 변수 조합을 탐색
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [15]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
# print(rgen)
rgen.rvs(10)

array([6, 4, 8, 4, 6, 3, 3, 6, 2, 2])

In [16]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 88,  94, 105,  96, 119, 108,  95, 102,  93, 100]))

In [17]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.72734975, 0.79868342, 0.39574255, 0.28038049, 0.44226536,
       0.03791437, 0.05480786, 0.02915843, 0.43044586, 0.13488118])

In [19]:
# 더 복잡한 매개 변수 조합을 탐색
params = {'min_impurity_decrease': uniform(0.001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),
          }
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 45, 'min_impurity_decrease': 0.001025419126744095, 'min_samples_leaf': 13, 'min_samples_split': 20}
0.8647301399274451
0.8561538461538462
